In [ ]:
import pandas as pd
from azure.cosmos import CosmosClient, PartitionKey
import json
import csv

### Access Database

In [ ]:
# Initialize the Cosmos client
cosmos_client = CosmosClient("https://prodccpdt.documents.azure.com:443/", "token")

database_name = 'ccpdt'
database = cosmos_client.get_database_client(database_name)

# Access the container
container_name = 'ccpdt'
container = database.get_container_client(container_name)

In [ ]:
#read the data
data = list(container.read_all_items())

In [ ]:
#normalize and convert to csv
df = pd.json_normalize(data)
df.to_csv('cc_data.csv', index=False)
df.head()

,type,status,companyName,companyType,numberOfEmployees,clients,offices,resourceUsers,logo,partitionKey,...,estimate.inclusions,estimate.exclusions,estimate.payments,estimate.zip_code,estimate.fax,estimate.custom_fields.Project Type,templateData.officeOwnerNextSteps,templateData.disciplines,deletedById,templateData.accountContributorNextSteps
0,0,0.0,"Project Cost Solutions, Inc.",1.0,10-20,[],[{'id': 'd2a33bab-811f-4910-bb8a-67931f8197e1'...,[{'id': '07b9dc23-c95e-4446-a043-7edb30de4717'...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",account,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,constructionCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,constructionCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disciplineSkill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disciplineSkill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get container properties
container_properties = container.read()
print(container_properties)
# Retrieve and print the partition key path
partition_key_path = container_properties['partitionKey']['paths']
print("Partition Key Path:", partition_key_path)

### Query user

In [ ]:
#Find the partition categories
query = "SELECT DISTINCT c.partitionKey FROM c"

# Execute the query
partition_keys = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

# Print the distinct partition keys
for pk in partition_keys:
    print(pk)


{'partitionKey': 'account'}
{'partitionKey': 'comment'}
{'partitionKey': 'constructionCategory'}
{'partitionKey': 'content:project'}
{'partitionKey': 'disciplineSkill'}
{'partitionKey': 'email'}
{'partitionKey': 'invitation'}
{'partitionKey': 'leger'}
{'partitionKey': 'proEst:snapshot'}
{'partitionKey': 'project'}
{'partitionKey': 'projectRate'}
{'partitionKey': 'projectType'}
{'partitionKey': 'projectTypeCategory'}
{'partitionKey': 'setting'}
{'partitionKey': 'user'}


In [ ]:
#user names and unique id
### Finds all the unique ID and roles
query = "SELECT c.givenName, c.familyName, c.id, c.primaryDiscipline, c.expertRole, c.expertType, c.userType FROM c WHERE c.partitionKey = 'user' "

# Execute the query
names = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

# Print the distinct IDs
for name in names:
    print(name)

# #convert to csv
# df = pd.json_normalize(names)
# df.to_csv('cc_names.csv', index=False)

### Query project

In [ ]:
#project status query
query = "SELECT c.statusHistory, c.accountId, c.officeId, c.id FROM c WHERE c.partitionKey = 'project' "

# Execute the query
projects = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

In [ ]:
# create csv
with open('project_status.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["account_id", "office_id", "project_id", "status_history"])

    # Each status history as a row
    for project in projects:
        for p in project['statusHistory']:
            line = json.dumps(p, separators=(',', ':'))
            writer.writerow([project['accountId'], project['officeId'], project['id'], line])

In [ ]:
#project estimates
query = "SELECT c.phases, c.id FROM c WHERE c.partitionKey = 'project' "

# Execute the query
projects = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

In [ ]:
# create csv for estimates
with open('project_estimates.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["project_id", "phase_type", "phase_status", "estimates_id", "created_date", "updated_date", "discipline", "estimate_status", "effort_hours", "manual_hours", "difficulty" ])

    # Each status history as a row
    for project in projects:
        for phase in project['phases']:
            if 'estimates' in phase:
                for est in phase['estimates']:
                    line = [
                        project['id'],
                        phase['phaseType'],
                        phase['status'],
                        est.get('id', ''),
                        est.get('createdAt', ''),
                        est.get('updatedAt', ''),
                        est.get('discipline', ''),
                        est.get('status', ''),
                        est.get('levelOfEffortHours', ''),
                        est.get('manualLevelOfEffortHours', ''),
                        json.dumps(est.get('difficulty', []), separators=(',', ':'))
                    ]
                    writer.writerow(line)

In [ ]:
# create csv for labor hours
with open('labor_hours.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["project_id", "labor_status", "user_id", "skills", "hours"])

    # Each status history as a row
    for project in projects:
        for phase in project['phases']:
            if 'estimates' in phase:
                for est in phase['estimates']:
                    if 'labor' in est:
                        for labor in est['labor']:
                            line = [
                                project['id'],
                                labor['status'],
                                labor.get('userId', ''),
                                labor.get('skills', ''),
                                json.dumps(labor.get('hours', []), separators=(',', ':'))
                            ]
                            writer.writerow(line)

In [ ]:
df_labor = pd.read_csv('labor_hours.csv')
df_user = pd.read_csv('cc_names.csv')
user_hours = df_labor.merge(df_user, on='user_id', how='left')
user_hours.to_csv('user_hours.csv', index=False)